In [ ]:
!pip install pyspark findspark nltk

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re 
import string

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true') #трекер, чтобы возвращать ресурсы
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Как проще всего создать RDD? Вызвать метод и передать ему нужный объект

In [ ]:
first_rdd  = sc.parallelize(range(1000000))

In [ ]:
first_rdd.getNumPartitions()

А можно ли менять количество партиций? Да, для этого есть два метода: repartition() и coalesce(). Первый используется для увеличения и уменьшения количества партиций, второй только для снижения, прчем coalesce будет работать эффективнее. Много партиций - дольше будет считаться, но если данных много, то обязательно нужно

repartition()` всегда приводит к равномерному перераспределению данных, что ведет к shuffle. Если Вы уменьшаете число партиций, то стоит использовать `coalesce()`, который может избежать shuffle

In [ ]:
first_rdd = first_rdd.repartition(5)
print(first_rdd.getNumPartitions())

In [ ]:
first_rdd = first_rdd.repartition(2)
print(first_rdd.getNumPartitions())

In [ ]:
first_rdd = first_rdd.coalesce(1)
print(first_rdd.getNumPartitions())

In [ ]:
a = %time first_rdd.sum()

Посмотрим на время выполнения для разного числа партиций

In [ ]:
result = []
first_rdd  = sc.parallelize(range(5000000))
for partition in range(1, 30):
    first_rdd = first_rdd.repartition(partition)
    time = %timeit -o  -n 1 -r 1 first_rdd.sum()
    time = time.best
    result.append((partition, time))

In [ ]:
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [ ]:
plt.plot([res[1] for res in result])

А почему лучший результат при небольшом количестве партиций?

In [ ]:
import multiprocessing

multiprocessing.cpu_count()

Из list также можно создавать RDD

In [ ]:
bad_list = [1, 2, 3, 'a', 10, 'b']

In [ ]:
bad_list_rdd = sc.parallelize(bad_list)

In [ ]:
bad_list_rdd.collect()

Еще можно создать RDD через textFile и wholeTextFiles

In [ ]:
text_rdd = sc.textFile('spark_text.txt')

In [ ]:
text_rdd.take(1)

wholeTextFiles создает PairRDD в формате key-value, где ключ - имя файла, а значения - то, что находистя в файле. Имена файлов считываются из папки через wholeTextFiles

In [ ]:
dirPath = 'files'
os.mkdir(dirPath)
with open(os.path.join(dirPath, "1.txt"), "w") as file1:
    _ = file1.write("[1 2 3]")
with open(os.path.join(dirPath, "2.txt"), "w") as file2:
    _ = file2.write("[4 5 6]")
textFiles = sc.wholeTextFiles(dirPath)

In [ ]:
textFiles

In [ ]:
textFiles.collect()

У RDD есть стандартно 2 типа методов - actions и transformations

**Actions**

Начнем с actions, то есть того, что заставит посчитать

In [ ]:
first_rdd = first_rdd.coalesce(2)

In [ ]:
first_rdd.sum()

In [ ]:
first_rdd.min(), first_rdd.max()

In [ ]:
first_rdd.first()

In [ ]:
first_rdd.take(2)

In [ ]:
first_rdd.count()

In [ ]:
a = first_rdd.collect()

In [ ]:
first_rdd.saveAsTextFile, first_rdd.saveAsPickleFile

In [ ]:
first_rdd.reduce(lambda x, y: x + y)

Если нужно получить небольшое число записей на драйвер и, при этом, сохранить распределение, то лучше сделать выборку

In [ ]:
first_rdd.takeSample(withReplacement=False, num=5, seed=5757)

**Transformations**

Это просто трансформации, которые не будут вычисляться до вызова actions

In [ ]:
a = sc.parallelize([1, 2, 3])
b = sc.parallelize([2, 3, 4])

In [ ]:
c = a + b

In [ ]:
c.collect()

filter

In [ ]:
text_rdd.count()

In [ ]:
text_rdd.filter(lambda x: x != '').count()

In [ ]:
text_rdd = text_rdd.filter(lambda x: x != '')

map

In [ ]:
stop_words = stopwords.words("english")
stop_words = set(stop_words)

In [ ]:
def mapper_text(text):
    clean_text = re.sub(rf"[{string.punctuation}]", "", text)
    words = nltk.word_tokenize(clean_text)
    words_with_value = [(word.lower(), 1) for word in words 
                        if word not in stop_words]
    return words_with_value

In [ ]:
text_rdd.map(mapper_text).take(1)

flatMap

Попробуем применить map и flatMap

In [ ]:
text_rdd.map(mapper_text).count()

In [ ]:
text_rdd.flatMap(mapper_text).count()

Как так?

In [ ]:
text_rdd.map(mapper_text).map(len).sum()

вроде понятно что случилось, но давайте на игрушечном примере

In [ ]:
simple_example = sc.parallelize([[1, 2, 3], [2, 3, 4], [4, 5, 6]])

In [ ]:
def pow_elements(elements):
    return [x**2 for x in elements]

In [ ]:
simple_example.map(pow_elements).collect()

In [ ]:
simple_example.flatMap(pow_elements).collect()

groupByKey

In [ ]:
text_rdd = text_rdd.flatMap(mapper_text)

In [ ]:
text_rdd.groupByKey().mapValues(len).collect()

sortByKey

In [ ]:
text_rdd.groupByKey().mapValues(len).sortByKey().collect()

И так на самом деле много методов, но предалагаю написать подсчет частоты слов и сделаем это в стиле программ на java

In [ ]:
text_rdd = sc.textFile('spark_text.txt')

In [ ]:
result = text_rdd.filter(lambda x: x != '')\
                 .flatMap(mapper_text)\
                 .groupByKey()\
                 .mapValues(len)\
                 .sortBy(lambda x: x[1], ascending=False)\
                 .collect()

In [ ]:
result[:10]

Забыли про reduceByKey

In [ ]:
text_rdd.filter(lambda x: x != '')\
        .flatMap(mapper_text)\
        .reduceByKey(lambda x, y: x + y)\
        .sortBy(lambda x: x[1], ascending=False)\
        .collect()[:10]

Стоит заметить, что `groupByKey()` предполагает перемещение всех записей с одним ключом на один экзекьютор. В случае очень скошенных распределений это может привести к падению экзекьютора с OOM. Поэтому всегда при группировках стоит подумать об использовании `reduceByKey()`.

Так, на лекции было что-то про count, который не делает shuffle да и вообще можно проще написать?

In [ ]:
def mapper_text_simple(text):
    clean_text = re.sub(rf"[{string.punctuation}]", "", text)
    words = nltk.word_tokenize(clean_text)
    words = [word.lower() for word in words 
                        if word not in stop_words]
    return words

In [ ]:
result = text_rdd.filter(lambda x: x != '')\
                 .flatMap(mapper_text_simple)\
                 .countByValue()
result = sorted(result.items(), key=lambda x: x[1], reverse=True)
print(result[:10])

**Join'ы**

Тут просто на игрушечном примере пощупаем данную операцию

In [ ]:
rdd_a = sc.parallelize([
                        ('a', [1, 2]),
                        ('b', [2, 4])])

rdd_b = sc.parallelize([
                        ('a', [10]),
                        ('c', [11])])

In [ ]:
rdd_a.join(rdd_b).collect()

In [ ]:
rdd_a.leftOuterJoin(rdd_b).collect()

In [ ]:
rdd_a.fullOuterJoin(rdd_b).collect()

**Домашнее задание 1**

Посчитать количество рейтингов больше 4 для каждого фильма и вывести фильмы в порядке убывания количества этих оценок (можно вывести топ 10)

Файл можете взять из прошлого домашнего задания + сохраните результат на диск

**Домашнее задание 2**

В этом задании у вас есть файл с обученным word2vec на произведении Достоевского Преступление и наказание. Файл - list, каждый элемент которого слово и его вектор в формате ('word', [vector]). Необходимо для каждого слова собрать список его top 10 похожих слов по косинусной метрике
Результат также сохраните на диск и выведите синонимы для слова 'топор' и 'деньга'.
Файл в пикле, так что для считывания воспользуйтесь не spark, можете взять любимый pandas

Файл не очень уж и маленький, рекомендую сначала написать код на кошках/собачках, а потом уже на всем, так как ядра 2, считаться будет очень долго.
В качестве одного из вариантов можете рассмотреть метод cartesian